In [ ]:

!pip install ultralytics opencv-python matplotlib


import cv2
import os
import json
import csv
from datetime import datetime
from ultralytics import YOLO


os.makedirs("resultados", exist_ok=True)


modelo = YOLO("yolov8n.pt")

cap = cv2.VideoCapture(0)
ret, frame = cap.read()
cap.release()

if not ret:
    raise Exception("Error al capturar imagen desde webcam.")

resultados = modelo.predict(frame)[0]

anotada = resultados.plot()

nombre_img = "resultados/deteccion.png"
cv2.imwrite(nombre_img, anotada)

detalles = []
timestamp = datetime.now().isoformat()

for box in resultados.boxes:
    clase = resultados.names[int(box.cls[0])]
    conf = float(box.conf[0])
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    detalles.append({
        "class": clase,
        "confidence": round(conf, 2),
        "x": x1,
        "y": y1,
        "w": x2 - x1,
        "h": y2 - y1
    })

data_json = {
    "timestamp": timestamp,
    "objects": detalles
}
with open("resultados/deteccion.json", "w") as f:
    json.dump(data_json, f, indent=2)

with open("resultados/resumen.csv", "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["timestamp", "clase", "confianza", "x", "y", "w", "h"])
    for obj in detalles:
        writer.writerow([
            timestamp,
            obj["class"],
            obj["confidence"],
            obj["x"],
            obj["y"],
            obj["w"],
            obj["h"]
        ])

print("✅ Exportación completada:")
print("- Imagen:", nombre_img)
print("- JSON: resultados/deteccion.json")
print("- CSV: resultados/resumen.csv")
